In [3]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import cv2
import pathlib
import torch
import torchvision.transforms as transforms
import cv2

DATA_DIR = "facial_expressions/data"
IMAGE_DIR = "facial_expressions/images"

legend = pd.read_csv(f'{DATA_DIR}/legend.csv')
legend['emotion'] = legend['emotion'].str.lower()
image_paths = list(pathlib.Path(IMAGE_DIR).glob('*.jpg'))
images = [plt.imread(str(path)) for path in image_paths]

In [4]:
import torch
data = pd.DataFrame({"path": [p.name for p in image_paths], "image": images})
data = data.join(legend.set_index('image'), on='path')

In [5]:
tensor_img = torch.Tensor(data['image'])
tensor_label = torch.Tensor(data['emotion'])

/tmp/ipykernel_34462/2838368823.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  tensor_img = torch.Tensor(data['image'])


ValueError: expected sequence of length 350 at dim 1 (got 134)

In [12]:
import numpy as np
a = np.array(images)

/tmp/ipykernel_34462/622740629.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.array(images)


In [14]:
convert_tensor = transforms.ToTensor()
convert_tensors = np.vectorize(convert_tensor)
ts = [convert_tensor(img) for img in images]
tensor_big = torch.Tensor(ts)

ValueError: only one element tensors can be converted to Python scalars

In [15]:
min((x.shape for x in images))import torch
df = pd.DataFrame({"path": [p.name for p in image_paths], "image": images}, dtype='')
df.join(data.set_index('image'), on='path')

(24, 25, 3)

In [16]:
max((x.shape for x in images))

(536, 356, 3)

In [17]:
from torch.utils.data import Dataset
import pandas as pd
import os
from torchvision.io import read_image

class FacialExpression(Dataset):
  def __init__(self, annotations_file=f'{DATA_DIR}/legend.csv', img_dir=IMAGE_DIR, transform=None, target_transform=None):
    self.img_labels = pd.read_csv(annotations_file)
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = read_image(img_path)
    label = self.img_labels.iloc[idx, 1]
    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
    return image, label

In [18]:
fe = FacialExpression()